# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [1]:
# import findspark
# findspark.init()

import pyspark  # only run after findspark.init()
from pyspark.sql import SparkSession

# May take awhile locally
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

22/10/08 23:50:32 WARN Utils: Your hostname, masoud-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.7.139 instead (on interface wlp2s0)
22/10/08 23:50:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/08 23:50:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/08 23:50:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
You are working with 1 core(s)


## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [2]:
df = spark.read.csv("Datasets/googleplaystore.csv", inferSchema=True, header=True)

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [3]:
df.limit(3).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up


As well as the schema to make sure all the column types were correctly infered

In [4]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [5]:
df.count()

10841

Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [21]:
from pyspark.sql.types import IntegerType, FloatType

df = (
    df.withColumn("Rating", df["Rating"].cast(FloatType()))
    .withColumn("Reviews", df["Reviews"].cast(IntegerType()))
    .withColumn("Price", df["Price"].cast(IntegerType()))
)
print(newdf.printSchema())
df.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [22]:
# Create a temporary view of the dataframe
df.createOrReplaceTempView("tempview")

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [23]:
spark.sql("SELECT * FROM tempview WHERE Rating > 4.1").limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
1,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
3,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
4,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up


## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the COMICS category and the Rating is above 4.5.

In [24]:
res_1 = spark.sql(
    "SELECT App, Rating FROM tempview WHERE Rating > 4.5 AND Category='COMICS'"
)
res_1.limit(5).toPandas()

,App,Rating
0,Manga Master - Best manga & comic reader,4.6
1,GANMA! - All original stories free of charge f...,4.7
2,Röhrich Werner Soundboard,4.7
3,Unicorn Pokez - Color By Number,4.8
4,Manga - read Thai translation,4.6


## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [25]:
spark.sql(
    "SELECT Category, sum(reviews) AS Total_Reviews FROM tempview GROUP BY Category ORDER BY Total_Reviews DESC"
).limit(1).toPandas()

,Category,Total_Reviews
0,GAME,1585422349


## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [26]:
spark.sql("SELECT App, Reviews FROM tempview ORDER BY Reviews DESC").limit(1).toPandas()

,App,Reviews
0,Facebook,78158306


In [27]:
spark.sql("SELECT App, Reviews FROM tempview ORDER BY Reviews DESC").show(5)

+------------------+--------+
|               App| Reviews|
+------------------+--------+
|          Facebook|78158306|
|          Facebook|78128208|
|WhatsApp Messenger|69119316|
|WhatsApp Messenger|69119316|
|WhatsApp Messenger|69109672|
+------------------+--------+
only showing top 5 rows



## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [34]:
spark.sql("SELECT App FROM tempview WHERE LOWER(App) LIKE '%dating%'").show(5)

+--------------------+
|                 App|
+--------------------+
|Private Dating, H...|
|AMBW Dating App: ...|
|Zoosk Dating App:...|
|      OkCupid Dating|
|Match™ Dating - M...|
+--------------------+
only showing top 5 rows



## 6. Use SQL Transformer to display how many free apps there are in this list

In [35]:
# First we need to import SQL transformer
from pyspark.ml.feature import SQLTransformer

In [36]:
sqlTrans = SQLTransformer(statement="SELECT count(*) FROM __THIS__ WHERE Type = 'Free'")
sqlTrans.transform(df).show()

+--------+
|count(1)|
+--------+
|   10037|
+--------+



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [37]:
sqlTrans = SQLTransformer(
    statement="SELECT Genres, count(*) as Total FROM __THIS__ GROUP BY Genres ORDER BY Total DESC"
)
sqlTrans.transform(df).show(1)

+------+-----+
|Genres|Total|
+------+-----+
| Tools|  842|
+------+-----+
only showing top 1 row



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [38]:
sqlTrans = SQLTransformer(
    statement="SELECT App, Reviews FROM __THIS__ WHERE Genres = 'Tools' AND Reviews > 100"
)
sqlTrans.transform(df).show(10)

+--------------------+--------+
|                 App| Reviews|
+--------------------+--------+
|   Moto File Manager|   38655|
|              Google| 8033493|
|    Google Translate| 5745093|
|        Moto Display|   18239|
|      Motorola Alert|   24199|
|     Motorola Assist|   37333|
|Cache Cleaner-DU ...|12759663|
|  Moto Suggestions ™|     308|
|          Moto Voice|   33216|
|          Calculator|   40770|
+--------------------+--------+
only showing top 10 rows



## That's all folks! Great job!